# 8. 전이학습(Transfer Learning)

전이학습이란 기존의 잘 알려진 데이터 혹은 사전학습 된 모델을 업무 효율 증대나 도메인 확장을 위해 사용하는 학습을 의미한다. 따라서 전이학습은 인공지능 분야에서 매우 중요한 연구 중 하나이며 다양한 방법론들이 존재한다. 8강에서는 잘 학습 된 모델을 재사용하는 방법에 대해서 설명한다.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd/content/drive/MyDrive/Colab_Notebooks/deeplearningbro/pytorch

/content/drive/MyDrive/Colab_Notebooks/deeplearningbro/pytorch


## 8.1 GPU 연산 확인
빠른 병렬 처리를 위하여 CPU보다는 GPU연산이 사용된다. CUDA를 사용할 수 있는 NVIDIA 그래픽이 있다면 CUDA 버전과 torch 버전을 맞춰 본인의 컴퓨터를 세팅하도록 한다. 우리는 이런 수고스러움을 덜기 위해 무료로 GPU 연산을 제공하는 Google Colaboratory(이하 코랩)을 이용할 것이다. 코랩은 별도의 설치 없이 누구나 무료로 GPU를 사용할 수 있다.

In [5]:
# GPU vs CPU
# 현재 가능한 장치를 확인한다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## **8.2** CIFAR10 데이터 불러오기

In [6]:
# 데이터 불러오기 및 전처리 작업
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

# Class
#'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'

Files already downloaded and verified
Files already downloaded and verified


## **8.3** Pretrained model 불러오기
파이토치에서는 다양한 사전 학습 된 모델을 제공하고 있다.
https://pytorch.org/docs/stable/torchvision/models.html

In [7]:
# ResNet18 불러오기 
# pretrained=True를 하면 ResNet18 구조와 사전 학습 된 파라메타를 모두 불러온다.
# pretrained=False를 하면 ResNet18 구조만 불러온다.
# 모델과 텐서에 .to(device)를 붙여야만 GPU 연산이 가능하니 꼭 기입한다.

model = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# 모델의 구조를 보면 마지막 출력 노드(fc의 out_features)가 1000개라는 것을 알 수 있다.
# 이는 1000개의 클래스를 가진 ImageNet 데이터를 이용하여 사전학습 된 모델이기 때문이다. 
# 따라서 우리가 사용하는 CIFAR10 데이터에 맞게 출력층의 노드를 10개로 변경해야만 한다.

num_ftrs = model.fc.in_features # fc의 입력 노드 수를 산출한다. 512개
model.fc = nn.Linear(num_ftrs, 10) # fc를 nn.Linear(num_ftrs, 10)로 대체한다.
model = model.to(device)

In [10]:
# 출력층의 노드가 10개로 바껴있다.
# Linear(in_features=512, out_features=10, bias=True)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 8.4 손실함수와 최적화 방법 정의

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

## 8.5 사전학습 모델을 이용한 학습

In [12]:
for epoch in range(20):

    running_loss = 0.0
    for data in trainloader:
        
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    cost = running_loss / len(trainloader)        
    print('[%d] loss: %.3f' %(epoch + 1, cost))  

torch.save(model.state_dict(), './models/cifar10_resnet18.pth')      

print('Finished Training')

[1] loss: 1.216
[2] loss: 0.909
[3] loss: 0.826
[4] loss: 0.775
[5] loss: 0.740
[6] loss: 0.722
[7] loss: 0.699
[8] loss: 0.678
[9] loss: 0.665
[10] loss: 0.646
[11] loss: 0.634
[12] loss: 0.627
[13] loss: 0.618
[14] loss: 0.608
[15] loss: 0.601
[16] loss: 0.598
[17] loss: 0.591
[18] loss: 0.585
[19] loss: 0.583
[20] loss: 0.576
Finished Training


## 8.6 모델 평가

In [ ]:
# 모델을 불러올 땐 똑같은 구조가 되어야 함.

model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features # fc의 입력 노드 수를 산출한다. 512개
model.fc = nn.Linear(num_ftrs, 10) # fc를 nn.Linear(num_ftrs, 10)로 대체한다.
model = model.to(device)

model.load_state_dict(torch.load('./models/cifar10_resnet18.pth')) # (저장된, 학습된 모델의 파라미터를 덮어씌워준다 느낌)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

## 여기서 발견하는 transfer learning 전 후 성능 향상 요인
- 지금 사용하는 resnet18의 layer가 훨씬 깊음
- RandomCrop이라는 Augmentation
- Pretrained 모델 사용 (초깃값이 랜덤으로 정해진 것보다는 유사한 task의 pretrained된 모델의 파라미터로 시작하면 학습이 더 잘됨 )